# 전지적 독자 시점 댓글 감성분석(다중분류)

In [38]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [7]:
from keras.preprocessing.text import Tokenizer

In [8]:
keyword = {'사이다인':0,'빵터지는':1,'설레는':2,'어두운':3,'기다려지는':4,'그림체가 귀여운':5,'그림체가 디테일한':6,'잘생긴':7,'똑똑한':8,'먼치킨인':9,'게임물':10,'소름돋는':11,'판타지물':12,'고구마인':13}
label = ['사이다인','빵터지는','설레는','어두운','기다려지는','그림체가 귀여운','그림체가 디테일한','잘생긴','똑똑한','먼치킨인','게임물','소름돋는','판타지물','고구마인']

## 데이터 정리 및 train 과 test 데이터 생성

In [9]:
data = pd.read_csv('전독시 라벨링.csv', encoding='utf-8')

In [10]:
data.head()

,comment,label
0,도깨비 얼굴이 ㅎㄷㄷ,어두운
1,어 저 잼민인데 너무 재미있어요,기다려지는
2,와 존나 재밌는데 ㄹㅇ,기다려지는
3,나만 도깨비 첫인상 귀엽다고 생각함,그림체가 귀여운
4,와 ㅁㅊ,소름돋는


In [11]:
data['label'].replace('', np.nan, inplace=True)# 널값 제거하기 위해 빈 칸은 널값을 넣어주기
data = data.dropna(how = 'any') # Null 값이 존재하는 행 제거

In [12]:
data['new_label']= data['label'].apply(lambda v: keyword.get(v, v))

In [13]:
data

,comment,label,new_label
0,도깨비 얼굴이 ㅎㄷㄷ,어두운,3
1,어 저 잼민인데 너무 재미있어요,기다려지는,4
2,와 존나 재밌는데 ㄹㅇ,기다려지는,4
3,나만 도깨비 첫인상 귀엽다고 생각함,그림체가 귀여운,5
4,와 ㅁㅊ,소름돋는,11
...,...,...,...
722,오늘도 재미있네요독자 얼굴이,잘생긴,7
723,유중혁처럼 말해보기ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,빵터지는,1
724,잔인한건 너희들이겠지 역겨워,어두운,3
725,막컷 독자 되게 잘생겼다 누가 이 독자보고 오징어래,잘생긴,7


In [16]:
data['new_label'].unique()

array([ 3,  4,  5, 11,  9,  6, 10,  7, 12,  8,  1,  0, 13,  2],
      dtype=int64)

## =========================================
## 데이터 준비 

## 리스트 텐서로 변환 

In [17]:
stopwords = ['진짜', '존나', '좀', '정말', '왜', '그냥', '것', '개', '뭐', '사람', '놈', '너', '걍', '잘', '임', '제발', '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
from konlpy.tag import Okt


In [18]:
okt = Okt() 
X = []
for sentence in data['comment']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거 
    X.append(temp_X)



In [19]:
# 정수 인코딩
max_words = 15000
tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True) # 특수 문자를 제거 및 단어개수 최대값 설정
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X) 



In [20]:
print((X[:4]))

[[35, 29, 374], [200, 31, 760, 112, 6, 91], [4, 5, 70], [37, 88, 35, 761, 11, 58, 124]]


In [21]:
#Y =data['new_label'] 

In [22]:
Y = []
for i in data['new_label'] :
    Y.append(i)

In [23]:
print(type(Y))

<class 'list'>


In [24]:
print(type(X[0][0]))

<class 'int'>


In [25]:
maxlen = 30 
class_number = 14                                    # 분류할 클래스의 수

In [26]:
# 원-핫 인코딩
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
X = sequence.pad_sequences(X, maxlen=maxlen)
Y = to_categorical(Y, num_classes=class_number)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Flatten

In [39]:
# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42) # Y는 따로 처리하지 않고 넣음.

In [40]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

## =========================================
## 모델 학습

In [41]:
model = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 input_length를 지정합니다.
model.add(Embedding(10000, 8, input_length=maxlen))
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됩니다.

# 3D 임베딩 텐서를 (samples, maxlen * 8) 크기의 2D 텐서로 펼칩니다.
model.add(Flatten())

# 분류기를 추가합니다.
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 8)             80000     
_________________________________________________________________
flatten_4 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 241       
Total params: 80,241
Trainable params: 80,241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
14/14 [==============================] - 1s 20ms/step - loss: 0.6880 - acc: 0.5095 - val_loss: 0.6307 - val_acc: 0.9074
Epoch 2/10
14/14 [==============================] - 0s 6ms/step - loss: 0.6121 - acc: 0.9223 - val_loss: 0.5519 - val_acc: 0.9167
Epoch 3/10
14/14 [==============================] - 0s 7ms/step - loss: 0.5386 - acc: 0.9167 - val_loss: 0.4708 - val_a